# PATIENT to OMOP CDM

#### DUDAS DE CIERTOS MAPEOS

Respecto a las tablas de Observation, Measurement y Condition_occurence, ¿se mete en la misma fila de los observation_concept_id todos los valores de las observaciones? ¿O se crean varias filas?

Si se repite algún observation_concept_id ¿se eliminan? Pueden generarse multitud de datos de lo contrario.

#### LINKS RECOMENDADOS 

Para sacar el valor correspondiente del vocabulario de la columna de ethnicity_concept_id de la tabla PERSON en OMOP (entre otros):
https://inspiredata.network/etl/IDSR2OMOP/IDS2OMOP-Assay-Part-1-v1.0/measurement.html 

Valor del vocabulario de "condition_type_concept_id":
https://github.com/OHDSI/OMOP-Queries/blob/master/md/Condition_Occurence.md


In [55]:
import pandas as pd
import psycopg2
import random
import numpy as np
from datetime import datetime, timedelta

def get_random_value(val):
    if isinstance(val, tuple):
        return np.random.choice(val)
    return val

### Extracción de datos Patient 

A continuación, leemos nuestros datos y nos conectamos a nuestra base de datos de OMOP que hayamos desplegado de forma local con Docker.

In [56]:
# Leer datos del archivo CSV
df_patient_IDEA4RC = pd.read_csv("./IDEA4RC-data/patientsIDEA4RC.csv")
df_patient_IDEA4RC.head(5)

,id,Gender,Race,Birth year,Country of Residence,Smoking,Cigarettes pack years smoked during life,Alcohol,Height/weight (BMI),Charslon Comorbidity Index,...,McCune-Albright syndrome,Multiple osteochondromas,Neurofibromatosis type 1,Rothmund-Thomson syndrome,Werner syndrome,Retinoblastoma,Paget disease,Other Genetic syndrome WHO 2020,Occurrence of other cancer,Previous cancer treatment
0,2,8532,8527,0,4329169,1585856,903650,1586197,4245997,42538860,...,37117262,37396802,377252,4286355,4197821,4158977,75910,1340204,1340204,1340204
1,27,8532,8515,0,4329169,1585856,903650,1586197,4245997,42538860,...,37117262,37396802,377252,4286355,4197821,4158977,75910,1340204,1340204,1340204
2,3,8532,8515,0,4329169,1585856,903650,1586197,4245997,42538860,...,37117262,37396802,377252,4286355,4197821,4158977,75910,1340204,1340204,1340204
3,44,8507,8516,0,4329169,1585856,903650,1586197,4245997,42538860,...,37117262,37396802,377252,4286355,4197821,4158977,75910,1340204,1340204,1340204
4,14,8507,8516,0,4329169,1585856,903650,1586197,4245997,42538860,...,37117262,37396802,377252,4286355,4197821,4158977,75910,1340204,1340204,1340204


In [57]:
# Conexión a la base de datos OMOP CDM
conn = psycopg2.connect(
    dbname="omopdb",
    user="postgres",
    password="mysecretpassword",
    host="localhost",
    port="5432"
)

# Comando para hacer el mapeo (insercción de datos del IDEA4RC.csv a OMOP)
cur = conn.cursor()

### Patient to Person table

Para cada columna de patientsIDEA4RC.csv correspondiente a la tabla de person, haremos la transformación necesaria para su posterior mapeo.

In [58]:
person_mapping = ['id', "Gender", "Race", "Birth year", "Country of Residence"]

# Dataframe para columnas y datos necesarios no disponibles o no definidos en Idea4RC
df_tables = pd.DataFrame()
df_tables.index = range(len(df_patient_IDEA4RC))
df_tables['Ethnicity'] = [4087925 if random.random() > 0.5 else 0 for _ in range(len(df_tables))] #Este valor no está en el excel de IDEA4RC. Necesitamos incluirlo y verificar también si es ese número o si hay más

# #Conversión a objetos de las columnas de nuestro csv para poder ingestar los datos de forma correcta en la BD.
# df_tables['Ethnicity'] = df_tables['Ethnicity'].astype(object)

# for column in person_mapping:
#     df_patient_IDEA4RC[column] = df_patient_IDEA4RC[column].astype(object)

# Crear la consulta SQL para la inserción
sql = """
    INSERT INTO omopcdm.person (person_id, gender_concept_id, race_concept_id, year_of_birth, location_id, ethnicity_concept_id)
    VALUES (%s, %s, %s, %s, %s, %s)
"""

# Ejecutar la consulta SQL para el mapeo a OMOP con los valores de las columnas
cur.executemany(sql, zip(df_patient_IDEA4RC['id'], df_patient_IDEA4RC['Gender'], df_patient_IDEA4RC['Race'],
                         df_patient_IDEA4RC["Birth year"], df_patient_IDEA4RC["Country of Residence"], df_tables["Ethnicity"]))

# Confirmar la transacción
conn.commit()

# # Cerrar conexión
# cur.close()
# conn.close()

### Patient to Observation table

Para cada columna de patientsIDEA4RC.csv correspondiente a la tabla de observation, haremos la transformación necesaria para su posterior mapeo.

Hay que hacer incapié en que este mapeo no es como los demás. Por cada "id" =="person_id", necesitamos añadir tantas filas necesarias como valores en las columnas observations tenga ese paciente.

In [59]:
patient_column_observation_names = [
    'Smoking', 'Cigarettes pack years smoked during life', 'Alcohol', 'Comorbidity', 'Myocardial infarction',
    'Congestive heart failure', 'Peripheral vascular disease', 'Cerebrovascular accident (except hemiplegia)',
    'Dementia', 'Chronic pulmonary disease', 'Connective tissue disease', 'Ulcer', 'Mild liver disease',
    'Moderate to severe liver disease', 'Diabetes (without complications)', 'Diabetes with end organ damage',
    'Hemiplegia', 'Moderate to severe renal disease', 'Solid tumor (non metastatic)', 'Metastatic solid tumor',
    'Leukemia', 'Lymphoma', 'Multiple myeloma', 'AIDS', 'Eastern Cooperative Oncology Group performance status (ECOG PS) at diagnosis',
    'ECOG PS label', 'Karnofsy index label', 'No Genetic syndrome WHO 2020',
    'Other Genetic syndrome WHO 2020', 'Occurrence of other cancer', 'Previous cancer treatment']

#Para las columnas del excel en las que tengamos disponible un Vocabulary y Value as modifier, añadiremos al dataframe patients
#un ejemplo de posibles vocabularios que añadir a la columna value_as_concept_id en la tabla de OBSERVATION de OMOP

patient_column_observation_values = ['Smoking', 'Alcohol', 'Comorbidity', 'ECOG PS label', 'Karnofsy index label', 'No Genetic syndrome WHO 2020', 'Occurrence of other cancer' 'Previous cancer treatment']

observation_vocab_values_concept_id= { 
    'Smoking_val': (36309332, 45883458, 45879404),
    'Alcohol_val': (4074035, 4117706, 37204556, 4220362),
    'Comorbidity_val': 46235351,
    'ECOG PS label_val': 36303470, #Vocabulary = Variable as modifier
    'Karnofsy index label_val': 36303744, #Vocabulary = Variable as modifier
    'No Genetic syndrome WHO 2020_val': 4211787,
    'Occurrence of other cancer_val': 4266186,
    'Previous cancer treatment_val': (4273629, 4170755, 4121697, 42535584, 4295112)
}

for key, value in observation_vocab_values_concept_id.items():
    df_patient_IDEA4RC[key] = [get_random_value(value) for _ in range(len(df_patient_IDEA4RC['id']))]

#Añadimos columnas y datos necesarios no disponibles o no definidos en Idea4RC
df_tables["Observation_type"] = [32817 if random.random() > 0.5 else 0 for _ in range(len(df_patient_IDEA4RC))]
# df_tables["Observation_type"] = df_tables["Observation_type"].astype(object)

df_tables["Date"] = [datetime.now().date()] * len(df_patient_IDEA4RC)

df_patient_IDEA4RC = df_patient_IDEA4RC.astype(object)
df_tables = df_tables.astype(object)
# Crear una lista para almacenar los valores que se van a insertar. 
values_to_insert = []

for idx, row in df_patient_IDEA4RC.iterrows():
    person_id = row['id']
    date_value = df_tables.loc[idx]['Date'].strftime('%Y-%m-%d')  # Get the 'Date' value from df_tables
    observation_type_value = df_tables.loc[idx]['Observation_type']
    for column in patient_column_observation_names:
        if column in patient_column_observation_values:
            observation_concept = row[column]
            observation_value_column = column + '_val'
            observation_value = row[observation_value_column]
            values_to_insert.append((person_id, observation_concept, date_value, observation_type_value, observation_value))
        else:
            observation_concept = row[column]
            observation_value = None
            values_to_insert.append((person_id, observation_concept, date_value, observation_type_value, observation_value))
            
# Crear una lista desde 0 hasta el máximo (inclusive) para la columna de "observation_id" de OMOP,
# que obliga a tener un valor único por cada observación distinta.
observation_ids = list(range(len(values_to_insert) + 1)) 

# Crear la consulta SQL para la inserción
sql = """
    INSERT INTO omopcdm.observation (observation_id ,person_id, observation_concept_id, observation_date, observation_type_concept_id, value_as_concept_id)
    VALUES (%s, %s, %s, %s, %s,%s)
"""
# Combinar observation_ids con values_to_insert para poder hacer el insert de manera sencilla
data_to_insert = [(observation_id, *values) for observation_id, values in zip(observation_ids, values_to_insert)]

# Ejecutar la consulta SQL con los valores no nulos
with conn.cursor() as cur:
    cur.executemany(sql, data_to_insert)

# Confirmar la transacción
conn.commit()

# # Cerrar la conexión
# conn.close()

### Patient to Measurement table 

In [60]:
patient_column_measurement_names = ['Height/weight (BMI)','Charslon Comorbidity Index']

#Añadimos columnas y datos necesarios no disponibles o no definidos en Idea4RC
df_tables['Measurement_type'] = [32809 if random.random() > 0.5 else 0 for _ in range(len(df_tables))]

#Conversión a objetos de las columnas de nuestro csv para poder ingestar los datos de forma correcta en la BD.
df_tables["Measurement_type"] = df_tables["Measurement_type"].astype(object)

# Crear una lista para almacenar los valores que se van a insertar
values_to_insert = []

for _, row in df_patient_IDEA4RC.iterrows():
    person_id = row['id']
    date_value = df_tables.loc[_]['Date'].strftime('%Y-%m-%d')  # Obtener el valor de 'Date' del DataFrame df_tables. Si no se hace así, figura como datetime?
    measurement_type_value = df_tables.loc[_]['Measurement_type'] 
    for column in patient_column_measurement_names:
        measurement_value = row[column]
        values_to_insert.append((person_id, measurement_value, date_value, measurement_type_value))
        
# Crear una lista desde 0 hasta el máximo (inclusive) para la columna de "observation_id" de OMOP,
# que obliga a tener un valor único por cada observación distinta.
measurement_ids = list(range(len(values_to_insert) + 1))  

# Crear la consulta SQL para la inserción
sql = """
    INSERT INTO omopcdm.measurement (measurement_id ,person_id, measurement_concept_id, measurement_date, measurement_type_concept_id)
    VALUES (%s, %s, %s, %s, %s)
"""
# Combinar measurement_ids con values_to_insert para hacer la insercción de datos de manera rápida
data_to_insert = [(measurement_id, *values) for measurement_id, values in zip(measurement_ids, values_to_insert)]

# Ejecutar la consulta SQL con los valores no nulos
with conn.cursor() as cur:
    cur.executemany(sql, data_to_insert)

# Confirmar la transacción
conn.commit()

# # Cerrar la conexión
# conn.close()

### Patient to Condition_occurrence table 

In [61]:
# Convertir todos los valores a tipos nativos de Python antes de la inserción
patient_column_condition_names = ['Olliers disease','Maffuci syndrome','Li-Fraumeni syndrome','McCune-Albright syndrome','Multiple osteochondromas','Neurofibromatosis type 1','Rothmund-Thomson syndrome','Werner syndrome','Retinoblastoma','Paget disease']

for column in patient_column_condition_names:
    df_patient_IDEA4RC[column] = df_patient_IDEA4RC[column].astype(object)

    
df_tables['condition_type'] = [42894222 if random.random() > 0.5 else 0 for _ in range(len(df_tables))]
df_tables["condition_type"] = df_tables["condition_type"].astype(object)

# Crear una lista para almacenar los valores que se van a insertar
values_to_insert = []

for _, row in df_patient_IDEA4RC.iterrows():
    person_id = row['id']
    date_value = df_tables.loc[_]['Date'].strftime('%Y-%m-%d')  # Obtener el valor de 'Date' del DataFrame df_tables
    condition_type_value = df_tables.loc[_]['condition_type'] 
    for column in patient_column_condition_names:
        condition_value = row[column]
        values_to_insert.append((person_id, condition_value, date_value, condition_type_value))
        

condition_ids = list(range(len(values_to_insert) + 1))  # Crear una lista desde 0 hasta el máximo (inclusive)

# Crear la consulta SQL para la inserción
sql = """
    INSERT INTO omopcdm.condition_occurrence (condition_occurrence_id ,person_id, condition_concept_id, condition_start_date, condition_type_concept_id)
    VALUES (%s, %s, %s, %s, %s)
"""
# Combinar condition_ids con values_to_insert
data_to_insert = [(condition_id, *values) for condition_id, values in zip(condition_ids, values_to_insert)]

# Ejecutar la consulta SQL con los valores no nulos
with conn.cursor() as cur:
    cur.executemany(sql, data_to_insert)

# Confirmar la transacción
conn.commit()

# # Cerrar la conexión
# conn.close()

In [62]:
df_tables.head(3)

,Ethnicity,Observation_type,Date,Measurement_type,condition_type
0,4087925,0,2024-06-06,0,42894222
1,4087925,32817,2024-06-06,32809,42894222
2,0,32817,2024-06-06,32809,42894222
